In [10]:
# 1 a

NUM_DAYS = 10

def run_simulation():
  # Record metrics per day
  daily_taxi_revenue = []
  daily_dispatcher_revenue = []
  daily_taxis_available = []

  for day in range(NUM_DAYS):
    # Run one day of the simulation
    total_taxi_revenue = 0
    total_dispatcher_revenue = 0

    taxis_available_each_hour = [0] * 24

    for hour in range(24):
      # Run one hour of simulation
      # Record total revenues
      total_taxi_revenue += sum(taxi.revenue for taxi in taxis)
      total_dispatcher_revenue += dispatcher.revenue

      # Record taxis available
      taxis_available_each_hour[hour] = len(taxis)

      daily_taxi_revenue.append(total_taxi_revenue / len(taxis))
      daily_dispatcher_revenue.append(total_dispatcher_revenue)
      daily_taxis_available.append(taxis_available_each_hour)

  # Calculate averages
  avg_daily_taxi_revenue = sum(daily_taxi_revenue) / NUM_DAYS
  avg_daily_dispatcher_revenue = sum(daily_dispatcher_revenue) / NUM_DAYS
  avg_daily_taxis_available = [sum(x) / NUM_DAYS for x in zip(*daily_taxis_available)]


In [16]:
# 1.b

#### Orininal Code _______________________________________________________________________________________________________
#       # TODO
#      # this function should build your route and fill the _path list for each new
#      # journey. Below is a naive depth-first search implementation. You should be able
#      # to do much better than this!
#      def _planPath(self, origin, destination, **args):
#          # the list of explored paths. Recursive invocations pass in explored as a parameter
#          if 'explored' not in args:
#             args['explored'] = {}
#          # add this origin to the explored list
#          # explored is a dict purely so we can hash its index for fast lookup, so its value doesn't matter
#          args['explored'][origin] = None
#          # the actual path we are going to generate
#          path = [origin]
#          # take the next node in the frontier, and expand it depth-wise
#          if origin in self._map:
#             # the frontier of unexplored paths (from this Node
#             frontier = [node for node in self._map[origin].keys() if node not in args['explored']]
#             # recurse down to the next node. This will automatically create a depth-first
#             # approach because the recursion won't bottom out until no more frontier nodes
#             # can be generated
#             for nextNode in frontier:
#                 path = path + self._planPath(nextNode, destination, explored=args['explored'])
#                 # stop early as soon as the destination has been found by any route.
#                 if destination in path:
#                    # validate path
#                    if len(path) > 1:
#                       try:
#                           # use a generator expression to find any invalid nodes in the path
#                           badNode = next(pnode for pnode in path[1:] if pnode not in self._map[path[path.index(pnode)-1]].keys())
#                           raise IndexError("Invalid path: no route from ({0},{1}) to ({2},{3} in map".format(self._map[path.index(pnode)-1][0], self._map[path.index(pnode)-1][1], pnode[0], pnode[1]))
#                       except StopIteration:
#                           pass
#                    return path
#          # didn't reach the destination from any reachable node
#          # no need, therefore, to expand the path for the higher-level call, this is a dead end.
#          return []
#### Orininal Code _______________________________________________________________________________________________________



import heapq
import math

# Graph of taxi network
graph = {}

def build_graph(map_data):
  for node, neighbors in map_data.items():
    graph[node] = {}
    for neighbor, cost in neighbors.items():
      graph[node][neighbor] = cost
  return graph

def euclidean_distance(pos1, pos2):
  x1, y1 = pos1
  x2, y2 = pos2
  return math.sqrt((x2 - x1)**2 + (Y2 - y1)**2)

def _planPath(origin, destination, map_data):
  graph = build_graph(map_data)
  # A* search / Priority queue
  queue = []
  heapq.heappush(queue, (0, origin)) # f_score, node
  # Dict for visited nodes
  visited = {origin: None}
  # While queue not empty
  while queue:

    # Continously extract lowest cost node for queue
    f_score, node = heapq.heappop(queue)
    # Check if at destination
    if node == destination:
      return reconstruct_path(destination)

    for neighbor in graph[node]:
      if neighbor not in visited:
        g_score = f_score + graph[node][neighbor]

        # Heuristic estimates remaining cost
        h_score = euclidean_distance(neighbor, destination)
        f_score = g_score + h_score

        # Prioritize closer nodes
        heapq.heappush(queue, (f_score, neighbor))

        # Avoid redundant exploaration
        visited[neighbor] = node

  return None

def reconstruct_path(current):

  path = []
  while visited[current] is not None:
    path.append(current)
    current = visited[current]
  return path[::1]





### Alternative _________________________________________________________________
#    def _planPath(self, origin, destination):
#
#        # Build graph of road network - Construct graph
#        graph = {}
#        for node, edges in self._map.items():
#            neighbors = {neighbor: cost for neighbor, cost in edges.values()}
#            graph[node] = neighbors
#
#        # A* search / Priority queue
#        queue = []
#        heapq.heappush(queue, (0, origin))
#
#        # Dict for visited nodes
#        visited = {origin: None}
#
#        # While queue not empty
#        while queue:
#
#            # Get node with lowest f-score
#            f, node = heapq.heappop(queue)
#
#            # Check if at destination
#            if node == destination:
#
#                # Construct path backward and return
#                path = []
#                while node is not None:
#                    path.append(node)
#                    node = visited[node]    ## came_from or visited
#                return path[::-1]

#           # Get neighbors
#            for neighbor, cost in graph[node].items():
#                if neighbor not in visited
#                # Calculate heuristics
#                g = f + cost
#                h = euclidean_distance(neighbor, destination)
#                f = g + h

#                # Add to queue
#                heapq.heappush(queue, (f, neighbor))
#                #Mark as visited
#                visited[neighbor] = node    ## came_from or visited
#
#        #No path found
#        return []
### Alternative _________________________________________________________________



In [9]:
# 1 c

import random
import logging

logging.basicConfig(level=logging.INFO)
    # This function plans a path from an original to destination probabilistically
    # by modelling uncertainty in edge travel times.
    # It builds a graph fromthe map where each edge has an expeted travel time.
    # It then adds Gaussian noise to model uncertainty, simulating real world variability in times.

    # A* search is then run on this probabilistic graph to find the best path according to expected travel times.
    # This provides a probable optimal path rather than a certainly optimal one.

class Node:
  def __init__(self,name):
    self.name = name
    self.expected_time = None
    self.actual_time = None
    self.neighbors = {}

    # Probabilistic path planning
def _planPath(map, origin, destination):
  validate_input(map, origin, destination)
  logging.info("Planning path from %s to %s", origin, destination)

       # Build graph by adding noise to expected travel times
  graph = build_probabilistic_graph(map)
  logging.debug("Graph built with %d nodes", len(graph))
       # A* search on graph
  path = a_star_search(graph, origin, destination)

  if path:
    logging.info("Path found: %s", path)
    return path

    logging.info("No path found")
    return None

def validate_input(map, origin, destination):
  if not map:
    raise ValueError("Invalid Map")
  if origin not in map:
    raise ValueError("Invalid Origin")
  if destination not in map:
    raise ValueError("Invalid Destination")

        # Build a prababilistic graph by adding Gaussian noise
def build_probabilistic_graph(map):
  graph = {}

  for name, neighbors in map.items():
    node = Node(name)
    for next, expected in neighbors.items():

      neighbor = Node(next)

      # Add uncertainty
      deviation = random.gauss(0, expected/4)
      actual = max(expected + deviation, 0)

      neighbor.expected_time = expected
      neighbor.actual_time = actual

      node.neighbors[neighbor] = actual

  graph[node] = node.neighbors

  logging.debug("Graph built with %d nodes", len(graph))

  return graph



01 Taxis.py - Potential improvement areas for the taxi.py code:
- Planning Path
    - _planPath() could be improved by implementing a search algorithm like A*
    to find optimal paths considering distance and traffic.
- Bid optimisation
- State tracking
- Messaging
- Testing & Evaluation
    - We may compare performance of different path finding and bidding strategies to justify design decisions.

In [ ]:
# 01 Taxis.py - Code block for 3 - a - _bidOnFare() function modification


# 3. In the last part you will update the bidding routines on the taxi and possibly the dispatcher.
# a)  Modify the taxis' _bidOnFare function to maximise its expected return.
# Consider the deterministic time-to-location case with no traffic and the probabilistic case where there is traffic.
# Analyse actual results against expected results for at least 3 trials. (10%)


import statistics
from statistics import mean

# Track expected outcomes for analysis
expected_returns = []

def _bidOnFare(self, fare):

  origin = fare.origin
  destination = fare.destination
  # Deterministic travel time
  t_det = self._estimateTravelTime(origin, destination)
  # Probabilistic travel times
  t_prob = self._estimateProbabilisticTravelTime(origin, destination)
  # Expected fare price
  price = fare.price
  # Expected profits for deterministic and probabilistic cases
  p_det = price - t_det
  p_prob = []

  for t in t_prob:
    p = price - t
    p_prob.append(p)

  # Weighted expected profit
  p_exp = sum(p*p_prob) / len(p_prob)

  # Only bid if expected profit is positive
  if p_exp > 0:

    # Bid price to maximize expected return
    bid_price = p_exp + bidding_overhead

    # Track expected return
    expected_returns.append(p_exp)

    return True

  return False

NUM_TRIALS = 10

def run_simulation():
  pass

# After multiple trial
for trial in range(NUM_TRIALS):
  run_simulation()

trial = []

# Analyze actual vs expected returns
acutial = [trial.trial_return for trial in trials]

if expected_returns and actual:
  print("Expected return:", statistics.mean(expected_returns))
  print("Actual return:", statistics.mean(actual))
else:
  print("No data available")


In [ ]:
# 06 Dispatcher - Code block for 3 - c - _costFare() function modification

# Question 3C - You could also modify the dispatcher's _costFare function to maximise the probability
# that each fare will be transported (i.e. minimise the likelihood that a fare will cancel).
# This will involve probabilistic reasoning over both the fares and the taxis.
# Consider how such a bidding and dispatch system might be used in a commercial environment. (5%)

## ___Given part ____________________________________________________________________________________________________ ##
#      # TODO - improve costing
#      def _costFare(self, fare):
#          timeToDestination = self._parent.travelTime(self._parent.getNode(fare.origin[0],fare.origin[1]),
#                                                      self._parent.getNode(fare.destination[0],fare.destination[1]))
#          # if the world is gridlocked, a flat fare applies.
#          if timeToDestination < 0:
#             return 150
#          return (25+timeToDestination)/0.9
## _________________________________________________________________________________________________________________ ##



import random

def _costFare(self, fare):
  origin = fare.origin
  destination = fare.destination

  # Estimated travel time distributions
  etas = self._estimateETAs(origin, destination)

  # Taxi reliability score
  taxis = self._getTaxiScore()

  best_taxi = None
  min_risk = float('inf')

  for taxi in taxis:
    # Taxis individual ETA probability distribution
    eta_dist = etas[taxi]

  # Risk = Sum of probability * cancellation cost
  risk = 0
  for eta, prob in eta_dist:

    # Cancellation probability based on past probability
    cancel_prob = self._getCancelProb(taxi, eta)
    risk += prob * cancel_prob * 150

  # Track best risky taxi
  if risk < min_risk:
    min_risk = risk
    best_taxi = taxi

  # Match fare
  self._mathFare(fare, best_taxi)

  # Use taxi's individual ETA distribution
  if best_taxi:
    return random.choice(etas[best_taxi])

  # No taxis, use global distribution
  return random.choice(etas)


# Here is dispatcher's _costFare() using probabilistic reasoning,
# that how such a bidding and dispatch system might be used in a commercial environment:
# - I considered probabilistic ETA based on origin, destination. No traffic considered in code.
# - Factor taxi reliability has been considered, including track cancellation rate for each tai over time.
# --- There might be some other reliability factors can be considered such as accident, complains...
# - Monitoring and refine matching has been considered. In my code, I record outcomes like completion times.